In [10]:
import pickle
import numpy as np
import psycopg2 as py

conn = py.connect(
    "dbname = 'mimic' user = 'postgres' host = 'localhost' port='5432' password = 'p13240!'")

In [2]:
cur = conn.cursor()
cur.execute("SET search_path TO mimiciii;")

In [3]:
cur.execute("""select hadm_id from admissions""")
list_adm_id = cur.fetchall()

In [4]:
len(list_adm_id)

58976

In [6]:
# Dictionary of vitals with itemid sets
vital_itemids = {
    "SpO2": [646, 220277],
    "HR": [211, 220045],
    "RR": [618, 615, 220210, 224690],
    "SBP": [51, 442, 455, 6701, 220179, 220050],
    "DBP": [8368, 8440, 8441, 8555, 220180, 220051],
    "EtCO2": [1817, 228640],
    "Temp_F": [223761, 678],
    "Temp_C": [223762, 676],
    "TGCS": [198, 226755, 227013],
    "CRR": [3348],
    "FiO2": [2981, 3420, 3422, 223835],
    "Glucose": [807, 811, 1529, 3745, 3744, 225664, 220621, 226537],
    "pH": [780, 860, 1126, 1673, 3839, 4202, 4753, 6003, 220274, 220734, 223830, 228243],
}

# Urine output itemids as a separate set since it's fetched from outputevents
urine_output_itemids = [
    43647, 43053, 43171, 43173, 43333, 43347, 43348, 43355, 43365, 
    43373, 43374, 43379, 43380, 43431, 43519, 43522, 43537, 43576, 
    43583, 43589, 43638, 43654, 43811, 43812, 43856, 44706, 45304, 227519
]

data = []
for id in range(len(list_adm_id)):
    hadm_id = list_adm_id[id][0]
    print(id, hadm_id)
    vitals = []

    # Loop through the vital itemids and execute a single query per vital type
    for vital_name, itemids in vital_itemids.items():
        itemid_str = ','.join(map(str, itemids))
        # cur.execute(f"SELECT charttime, valuenum FROM chartevents WHERE hadm_id = %s AND itemid IN ({itemid_str}) ORDER BY charttime", [hadm_id])
        vitals.append(f"SELECT charttime, valuenum FROM chartevents WHERE hadm_id = {hadm_id} AND itemid IN ({itemid_str}) ORDER BY charttime")

    # Fetch urine output from outputevents separately
    urine_itemid_str = ','.join(map(str, urine_output_itemids))
    # cur.execute(f"SELECT charttime, VALUE FROM outputevents WHERE hadm_id = %s AND itemid IN ({urine_itemid_str}) ORDER BY charttime", [hadm_id])
    vitals.append(f"SELECT charttime, VALUE FROM outputevents WHERE hadm_id = {hadm_id} AND itemid IN ({urine_itemid_str}) ORDER BY charttime")

    # Append the vitals for this admission ID to the main data list
    data.append(vitals)


0 100001
1 100003
2 100006
3 100007
4 100009
5 100010
6 100011
7 100012
8 100014
9 100016
10 100017
11 100018
12 100019
13 100020
14 100021
15 100023
16 100024
17 100025
18 100028
19 100029
20 100030
21 100031
22 100033
23 100034
24 100035
25 100036
26 100037
27 100038
28 100039
29 100040
30 100041
31 100044
32 100045
33 100046
34 100047
35 100050
36 100052
37 100053
38 100055
39 100058
40 100059
41 100060
42 100061
43 100062
44 100063
45 100065
46 100066
47 100068
48 100069
49 100071
50 100072
51 100074
52 100075
53 100077
54 100078
55 100079
56 100081
57 100085
58 100087
59 100088
60 100091
61 100094
62 100095
63 100096
64 100098
65 100099
66 100102
67 100103
68 100104
69 100106
70 100108
71 100109
72 100112
73 100113
74 100114
75 100116
76 100117
77 100118
78 100119
79 100120
80 100121
81 100124
82 100125
83 100126
84 100130
85 100131
86 100132
87 100133
88 100135
89 100136
90 100137
91 100138
92 100139
93 100140
94 100141
95 100142
96 100143
97 100146
98 100147
99 100150
100 100151

In [11]:
cur.close()
conn.close()

In [7]:
len(data)

58976

In [12]:
data[0]

['SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (646,220277) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (211,220045) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (618,615,220210,224690) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (51,442,455,6701,220179,220050) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (8368,8440,8441,8555,220180,220051) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (1817,228640) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (223761,678) ORDER BY charttime',
 'SELECT charttime, valuenum FROM chartevents WHERE hadm_id = 100001 AND itemid IN (223762,676) ORDER BY charttime',
 'SELECT c

In [13]:
np_data=np.array(data)

In [14]:
np.save('sql_for_vitals.npy', np_data)